In [34]:
from konlpy.tag import Komoran

In [35]:
komoran = Komoran(userdic='/tmp/dic.txt')
print(komoran.morphs(u'우왕 코모란도 오픈소스가 되었어요'))
print(komoran.nouns(u'오픈소스에 관심 많은 멋진 개발자님들!'))
print(komoran.pos(u'혹시 바람과 함께 사라지다 봤어?'))

['우왕', '코', '모란', '도', '오픈', '소스', '가', '되', '었', '어요']
['오픈', '소스', '관심', '개발자']
[('혹시', 'MAG'), ('바람과 함께 사라지다', 'NNP'), ('보', 'VV'), ('았', 'EP'), ('어', 'EF'), ('?', 'SF')]


# 네이버 영화 리뷰 텍스트 분석

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [37]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_train.txt", sep="\t")

In [38]:
data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [40]:
data.isna().sum()

id          0
document    5
label       0
dtype: int64

In [41]:
data[data['document'].isna()==True]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [42]:
data = data.dropna()

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [44]:
data['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [45]:
import string

In [46]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [47]:
# 특수문자 제거 함수
def clean(x):
    cleaned = []
    for i in x:
        if i in string.punctuation:
            cleaned.append(i.replace(i, " "))
        else:
            cleaned.append(i)
    cleaned = ''.join(cleaned)
    cleaned = cleaned.replace("   "," ").replace("  "," ")
    return cleaned

In [48]:
clean('흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나')

'흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나'

In [49]:
data['document'] = data['document'].apply(clean)

In [50]:
data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [51]:
data.loc[4]

id                                                    6483659
document    사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
label                                                       1
Name: 4, dtype: object

In [52]:
data['doc_len'] = data['document'].apply(len)

In [53]:
data.head()

,id,document,label,doc_len
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,17
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,28
2,10265843,너무재밓었다그래서보는것을추천한다,0,17
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,26
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,61


In [54]:
data.groupby('label')['doc_len'].mean()

label
0    34.062498
1    33.109549
Name: doc_len, dtype: float64

In [55]:
test = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_test.txt",sep='\t')
test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [56]:
# float object는 결측값있다는거니까 제거
test.isna().sum()

id          0
document    3
label       0
dtype: int64

In [57]:
test=test.dropna()

In [58]:
test['document'] = test['document'].apply(clean)

In [59]:
test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데 왜 3D로 나와서 제 심기를 불편하게 하죠,0


# 문자를 숫자형 벡터로 변환해주는 라이브러리
* countvector izer: 단어의 빈도를 세어서 숫자로 변환해주는 것
* TfidfVectorizer: 자주 나오는 단어에 가중치를 주어서 숫자로 변환해 주는것

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [61]:
# from konlpy.tag import Komoran
# komoran = Komoran()
# def tw_tokenizer(text):
#     tokens_ko = komoran.morphs(test)
#     return tokens_ko

In [66]:
cv = CountVectorizer(lowercase=False)
cv.fit(data['document'])
X=cv.transform(data['document'])

In [67]:
X

<149995x293925 sparse matrix of type '<class 'numpy.int64'>'
	with 1074747 stored elements in Compressed Sparse Row format>

In [68]:
y = data['label']

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [70]:
X

<149995x293925 sparse matrix of type '<class 'numpy.int64'>'
	with 1074747 stored elements in Compressed Sparse Row format>

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.4, random_state = 7)

In [77]:
lr = DecisionTreeClassifier(max_depth=3, random_state = 7)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
print(accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

0.5143504783492783
              precision    recall  f1-score   support

           0       0.96      0.04      0.07     30158
           1       0.51      1.00      0.67     29840

    accuracy                           0.51     59998
   macro avg       0.73      0.52      0.37     59998
weighted avg       0.74      0.51      0.37     59998



In [64]:
tf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tf_vect2 = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tf_vect.fit(data['document'])
tfidf_matrix_train = tf_vect.transform(data['document'])
tf_vect2.fit(test['document'])
tfidf_matrix_test = tf_vect.transform(test['document'])

AssertionError: phrase input should be string, not <class 'pandas.core.frame.DataFrame'>